In [ ]:
# ingore_index part highly important
dataset = dataset.append(dataset_nov, ignore_index= 'True')

### [RQ1.2]

How many times, on average, a user views a product before adding it to the cart?

In [ ]:
gc.collect()

# Creating a new dataframe with grouping the data by user_id, user_session, product_id and counting the number of occurrences for event_type "view":
view_df = dataset.groupby([dataset.user_id, dataset.user_session, dataset.product_id, dataset[dataset.event_type=='view'].event_type]).event_type.count()

gc.collect()

# Creating a new dataframe with grouping the data by user_id, user_session, product_id and counting the number of occurrences for event_type "cart":
cart_df = dataset.groupby([dataset.user_id, dataset.user_session, dataset.product_id, dataset[dataset.event_type=='cart'].event_type]).event_type.count()

gc.collect()

# Merging view_df and cart_df based on the same features which are user_id, user_session and product_id
view_cart_df = pd.merge(view_df, cart_df, how ='inner', on =['user_id', 'user_session', 'product_id'])

gc.collect()

# Finding the average number of views of a product before added to the cart
average_view_before_cart = view_cart_df.event_type_x.sum()/view_cart_df.event_type_y.sum()

print("Average of " + str(average_view_before_cart) + " times user views product before adding it to the cart.")

# Delete variable to free RAM
del view_df
del view_cart_df
del average_view_before_cart


### [RQ1.3]

What’s the probability that products added once to the cart are effectively bought?

In [ ]:
gc.collect()

# Creating a new dataframe with grouping the data by user_id, user_session, product_id and counting the number of occurrences for event_type "purchase":
purchase_df = dataset.groupby([dataset.user_id, dataset.user_session, dataset.product_id, dataset[dataset.event_type=='purchase'].event_type]).event_type.count()

gc.collect()

# Merging purchase_df and cart_df based on the same features which are user_id, user_session and product_id
# cart_df is created at RQ1.2
purchase_cart_df = pd.merge(purchase_df, cart_df, how ='inner', on =['user_id', 'user_session', 'product_id'])

gc.collect()

# Finding probability of the products that added to the card once are effectively bought
# Can be found as intersection of the bought and added to cart products over added to cart products based on the same user_session, user_id and product_id
probability_bought_given_added_to_cart = len(purchase_cart_df)/len(cart_df)

print("Probability of the products added once to the cart that are effectively bought is: " + str(probability_bought_given_added_to_cart))

# Delete variable to free RAM
del purchase_df
del purchase_cart_df
del probability_bought_given_added_to_cart
del cart_df

### [RQ1.5]

How much time passes on average between the first view time and a purchase/addition to cart?

In [ ]:
gc.collect()

# Creating a new dataframe by taking rows with event_type view
first_view_df = dataset[dataset.event_type=='view']

# Taking the first observed view event type for specific user_session, user_id and product_id
first_view_df = first_view_df.groupby(['user_session', 'user_id','product_id']).first()

gc.collect()

# Creating a new dataframe by tahing rows with event_type cart
cart_df = dataset[dataset.event_type=='cart']

gc.collect()

# Creating a new dataframe by tahing rows with event_type purchase
purchase_df = dataset[dataset.event_type=='purchase']

gc.collect()

# Constructing a new dataframe with using first_view_df and cart_df dataframes
# based on the same user_session, user_id, and product_id
time_elapse_between_first_view_and_cart_events=pd.merge(first_view_df, cart_df, how ='inner', on =['user_session', 'user_id', 'product_id'])

# Adding a new column in time_elapse_between_first_view_and_cart_events dataframe 
# to show time elapse between view event_type and cart event_type
time_elapse_between_first_view_and_cart_events['time_elapse']=time_elapse_between_first_view_and_cart_events.event_time_y-time_elapse_between_first_view_and_cart_events.event_time_x

gc.collect()

# Summing the time_elapse values to reach the average value and keeping the number of instances in the dataframe
total_time_elapse_between_first_view_and_cart_events = time_elapse_between_first_view_and_cart_events.time_elapse.sum()
count_total_time_elapse_between_first_view_and_cart_events = len(time_elapse_between_first_view_and_cart_events)

gc.collect()

# Constructing a new dataframe with using first_view_df and purchase_df dataframes
# based on the same user_session, user_id, and product_id
time_elapse_between_first_view_and_purchase_events=pd.merge(first_view_df, purchase_df, how ='inner', on =['user_session', 'user_id', 'product_id'])

# Adding a new column in time_elapse_between_first_view_and_purchase_events dataframe 
# to show time elapse between view event_type and purchase event_type
time_elapse_between_first_view_and_purchase_events['time_elapse']=time_elapse_between_first_view_and_purchase_events.event_time_y-time_elapse_between_first_view_and_purchase_events.event_time_x

gc.collect()

# Summing the time_elapse values to reach the average value and keeping the number of instances in the dataframe
total_time_elapse_between_first_view_and_purchase_events = time_elapse_between_first_view_and_purchase_events.time_elapse.sum()
count_total_time_elapse_between_first_view_and_purchase_events = len(time_elapse_between_first_view_and_purchase_events)

gc.collect()

# Calculating the average time elapse between view and purchase/cart event_types
average_time_passes_between_first_view_and_purchase_or_cart_events = (total_time_elapse_between_first_view_and_cart_events + total_time_elapse_between_first_view_and_purchase_events)
average_time_passes_between_first_view_and_purchase_or_cart_events = average_time_passes_between_first_view_and_purchase_or_cart_events /(count_total_time_elapse_between_first_view_and_cart_events + count_total_time_elapse_between_first_view_and_purchase_events)

print('Average time passes on average between the first view and a purchase or cart events is: '+ str(average_time_passes_between_first_view_and_purchase_or_cart_events))

# Delete variable to free RAM
del first_view_df
del cart_df
del purchase_df
del time_elapse_between_first_view_and_cart_events
del total_time_elapse_between_first_view_and_cart_events
del count_total_time_elapse_between_first_view_and_cart_events
del time_elapse_between_first_view_and_purchase_events
del total_time_elapse_between_first_view_and_purchase_events
del count_total_time_elapse_between_first_view_and_purchase_events
del average_time_passes_between_first_view_and_purchase_or_cart_events


### [RQ2.0] 


What are the categories of the most trending products overall? For each month visualize this information through a plot showing the number of sold products per category. 

In [ ]:
gc.collect()

# Creating a new dataframe with grouping the data by category_code and product_id in order to find the category of the most trending products sorted in descending order
categories_of_most_trending_products = dataset[dataset.event_type == 'purchase'].groupby(['category_code', 'product_id']).product_id.count().sort_values(ascending=False)

# Listing the most 10 popular products with category_code and amount sold information
print("The most 10 popular products overall with category_code and amount sold informations' are: ")
print("-----------------------------------------------------------------------------------")
print(categories_of_most_trending_products.head(10))


# Visualising the most 10 popular products with category_code and amount sold information
plt.figure()
categories_of_most_trending_products.head(10).plot.bar()
plt.title('Categories and the product names of the most trending products overall')
plt.xlabel('category_code/product_id')
plt.ylabel('amount of products bought')
plt.show()

gc.collect()

# Finding the unique months in the dataset to visualise month specific popular products and category information
unique_months_in_dataset=dataset.event_time.dt.month.unique()

# Creating new dataframe for each month and implementing the same code as above 
for i in range(len(unique_months_in_dataset)):
    new_df=dataset[dataset.event_time.dt.month==unique_months_in_dataset[i]]
    popular_products = new_df[new_df.event_type == 'purchase'].groupby(['category_code', 'product_id']).product_id.count().sort_values(ascending=False)
    plt.figure()
    popular_products.head(10).plot.bar()
    plt.title('month: '+str(unique_months_in_dataset[i]))
    plt.xlabel('category_code/product_id')
    plt.ylabel('amount of products bought')

    plt.show()
    
# Creating new dataframe for each month and visualising the number of products bought per category
for i in range(len(unique_months_in_dataset)):
    new_df=dataset[dataset.event_time.dt.month==unique_months_in_dataset[i]]
    popular_products = new_df[new_df.event_type == 'purchase'].groupby('category_code').event_type.count().sort_values(ascending=False)
    plt.figure()
    popular_products.plot.bar()
    plt.title('month: '+str(unique_months_in_dataset[i]))
    plt.xlabel('category_code')
    plt.ylabel('amount of products bought')
    plt.show()

# Delete variable to free RAM
del categories_of_most_trending_products
del unique_months_in_dataset


### [RQ2.1] 


Plot the most visited subcategories.

In [ ]:
gc.collect()

# Creating new dataframe based on view event type and grouping by category_id to find the most popular subcategory
# Sorting in descending order
most_visited_subcategories = dataset[dataset.event_type == 'view'].groupby(["category_id", "event_type"]).event_type.count().sort_values(ascending=False)

# Visualising the bar plot of the most visited 10 subcategory
plt.figure()
most_visited_subcategories.head(10).plot.bar()
plt.title('10 most visited subcategories at the overall dataset')
plt.xlabel('category_id')
plt.ylabel('total view')
plt.show()

# Delete variable to free RAM
del most_visited_subcategories

### [RQ2.2] 


What are the 10 most sold products per category?

In [ ]:
gc.collect()

# Creating a dataframe event_type only purchase and grouping by category_code and product_id to extract the number of products sold per category
ten_most_sold = dataset[dataset.event_type=='purchase'].groupby(['category_code', 'product_id']).product_id.count()

# Because of the not sorted number of products per category, the following line reaches top 10 values 
ten_most_sold = ten_most_sold.groupby('category_code').nlargest(10)

# Visualising the 10 most sold products per category
ten_most_sold.plot.bar(figsize=(18,3))

# Delete variable to free RAM
del ten_most_sold

### [RQ3.0] 


For each category, what’s the brand whose prices are higher on average?

In [ ]:
gc.collect()

# Creating a new dataframe with notnull value of price, grouping by category_code and brand, taking the mean value of the products' price for each brand
highest_price_brand_per_category = dataset[dataset.price.notnull()].groupby(["category_code", "brand"]).price.mean()

# Grouping the created dataframe by category_code in order to find the highest mean value of the price
highest_price_brand_per_category = highest_price_brand_per_category.groupby('category_code').nlargest(1)

# Printing the final result
print(highest_price_brand_per_category)

# Delete variable to free RAM
del highest_price_brand_per_category

### [RQ3.1] 


Write a function that asks the user a category in input and returns a plot indicating the average price of the products sold by the brand.

In [ ]:
# Taking the user input of category name
if __name__ =='__main__':
    print('Please write a category name to have a plot of average price of products of this category')
    cat_cod=str(input())

gc.collect()

# Eliminating null values of price
not_null_prices=dataset[dataset.price.notnull()]

gc.collect()

# Creating a new dataframe which has category_code entered by user, grouping by category_code and brand and taking the mean value
avg_price_of_products_brands_category= not_null_prices[not_null_prices.category_code==cat_cod].groupby(['category_code', 'brand']).price.mean()

# Plotting the average price of the products sold by the brand in user specified category
plt.figure()
avg_price_of_products_brands_category.plot.bar(figsize=(18,3))
plt.title('Average price of the products sold by the brands of '+ cat_cod + ' category')
plt.xlabel('Category/Brand')
plt.ylabel('Average price')
plt.show()

# Delete variable to free RAM
del not_null_prices
del avg_price_of_products_brands_category

### [RQ3.2] 

Find, for each category, the brand with the highest average price. Return all the results in ascending order by price.

In [ ]:
gc.collect()

# Creating a dataframe with not null values of price, grouping by category_code and brand, taking the mean of price
expensive_brand_by_each_category=dataset[dataset.price.notnull()].groupby(['category_code', 'brand']).price.mean()

# For each category taking the highest average value of price corresponds to most expensive brand
expensive_brand_by_each_category=expensive_brand_by_each_category.groupby('category_code').nlargest(1)

# Sorting values in the ascending order
expensive_brand_by_each_category=expensive_brand_by_each_category.sort_values()

# Printing the result
print(expensive_brand_by_each_category)

# Delete variable to free RAM
del expensive_brand_by_each_category